In [1]:
import polars as pl
import polars.selectors as cs
from glob import glob

In [2]:
from composable.strict import map
from composable.object import obj

# Lecture 4.1 - Download and unzip files programmically.

In this lecture, we will

1. Use `glob` to get local paths, and
2. Load multiple files using a list comprehension.
3. Combine multiple dataframes using `pd.concat`.
4. Add information from the path to the dataframe.

## Loading multiple files using a list comprehension

To load multiple files, we
1. Use `glob` to find the paths all relevant files.
2. Use a list comprehension to load the data into a list of DataFrames. 

The basic structure of this process is as follows.

**Cell 1 - `glob` cell.**
```python
from glob import glob

(paths :=
    glob("...glob_pattern using ** and *...", recursive=True)
)
```

**Cell 2 - List comprehension cell.**
import polars as pl

```python
(dfs := 
  [pl.read_csv(p) 
   # Addition processing can go here as part of the dot-chained commands
   for p in paths
   # You can add an if statement here to filter files if needed
 ]
)
```

### Example - Loading and combining all Attendance Quiz files and adding a path column

Note that the `data/old_attendance_example` directory contains multiple CSV files which we want to load and combine.

#### Step 1 - Use `glob` to get all the paths

**Cell 1 - `glob` cell.**
```python
from glob import glob

(paths :=
    glob("...glob_pattern using ** and *...", 
         recursive=True                       # This allows ** to search any number of subdirectories
         )
)
```

**Note.** See creation of `all_csvs` above

In [ ]:
(all_csvs :=
 glob('./data/old_attendance_example/**/*.csv', 
      recursive=True # Toggle comment and see what happens
     )
)

['./data/old_attendance_example\\dsci494s7\\Attendance Quiz - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 1 - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 2 - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 3 - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 4 - User Attempts.csv',
 './data/old_attendance_example\\stat180s18\\Attendance Quiz - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Attendance Quiz - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 1 - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 2 - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 3 - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 4 - User Attempts.csv']

#### Step 2 - Use a comprehension to load each file and add a path column and combine with `pl.concat`

**Cell 2 - List comprehension cell.**
import polars as pl

```python
(dfs := 
  [pl.read_csv(p) 
   # Addition processing can go here as part of the dot-chained commands
   for p in paths
   # You can add an if statement here to filter files if needed
 ]
)
```

**Note.** This requires all columns to have the same name and type across all files and that those columns are in the same order.

#### Step 2 - Read each CSV and add the path

In [10]:
(all_tables :=
    [pl.read_csv(p)
       .with_columns(path = pl.lit(p))
     for p in all_csvs
     if 'Attendance Quiz' in p
    ]
) >> map(obj.head(2))


[shape: (2, 11)
 ┌────────────┬──────────┬───────────┬──────────┬───┬────────────┬────────────┬─────────┬───────────┐
 │ Org        ┆ UserName ┆ FirstName ┆ LastName ┆ … ┆ Attempt_St ┆ Attempt_En ┆ Percent ┆ path      │
 │ Defined ID ┆ ---      ┆ ---       ┆ ---      ┆   ┆ art        ┆ d          ┆ ---     ┆ ---       │
 │ ---        ┆ str      ┆ str       ┆ str      ┆   ┆ ---        ┆ ---        ┆ str     ┆ str       │
 │ i64        ┆          ┆           ┆          ┆   ┆ str        ┆ str        ┆         ┆           │
 ╞════════════╪══════════╪═══════════╪══════════╪═══╪════════════╪════════════╪═════════╪═══════════╡
 │ 14460432   ┆ au9747cp ┆ Jericho   ┆ Greer    ┆ … ┆ 2019-01-14 ┆ 2019-01-14 ┆ 100 %   ┆ ./data/ol │
 │            ┆          ┆           ┆          ┆   ┆ 14:00:00   ┆ 14:06:00   ┆         ┆ d_attenda │
 │            ┆          ┆           ┆          ┆   ┆            ┆            ┆         ┆ nce_examp │
 │            ┆          ┆           ┆          ┆   ┆            ┆

#### Step 3 - Union the tables - YOLO!!!!

Taking a chance that
1. All the columns are the same (type, name, etc.), and
2. The columns are in the same order.

In [13]:
(attendance_quizzes :=
 pl.concat(all_tables)
).head()

Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent,path
i64,str,str,str,i64,i64,i64,str,str,str,str
14460432,"""au9747cp""","""Jericho""","""Greer""",1,1,1,"""2019-01-14 14:00:00""","""2019-01-14 14:06:00""","""100 %""","""./data/old_attendance_example\dsci494s7\Attendance Quiz - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",2,1,1,"""2019-01-16 14:00:00""","""2019-01-16 14:08:00""","""100 %""","""./data/old_attendance_example\dsci494s7\Attendance Quiz - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",3,1,1,"""2019-01-18 14:00:00""","""2019-01-18 14:05:00""","""100 %""","""./data/old_attendance_example\dsci494s7\Attendance Quiz - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",4,1,1,"""2019-01-23 14:00:00""","""2019-01-23 14:06:00""","""100 %""","""./data/old_attendance_example\dsci494s7\Attendance Quiz - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",5,1,1,"""2019-01-25 14:00:00""","""2019-01-25 14:10:00""","""100 %""","""./data/old_attendance_example\dsci494s7\Attendance Quiz - User Attempts.csv"""


### Step 4 - Extract information from the path

Finally, we will use `polars` string methods to extract the course information from the path.


In [34]:
pl.Config.set_fmt_table_cell_list_len(200) # Make the list column output longer
pl.Config.set_fmt_str_lengths(200) # Make string column output longer

(attendance_quizzes_w_info :=
 attendance_quizzes

.with_columns(path=pl.col('path').str.replace_all(r'\\', '/'))  # Replace \ with /
    .with_columns(path_split=pl.col('path').str.split('/'))
.with_columns(course_info = pl.col('path_split').list.get(3),
               file_name = pl.col('path_split').list.get(-1),
              )
  .with_columns(program = pl.col('course_info').str.slice(0,4),
                course_number = pl.col('course_info').str.slice(4, 3),
                section = pl.col('course_info').str.slice(7),
                quiz_type = pl.col('file_name').str.split(' ').list.get(0),
                )
 .drop(cs.starts_with('path', 'file') | cs.ends_with('info'))

)

Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent,program,course_number,section,quiz_type
i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,str
14460432,"""au9747cp""","""Jericho""","""Greer""",1,1,1,"""2019-01-14 14:00:00""","""2019-01-14 14:06:00""","""100 %""","""dsci""","""494""","""s7""","""Attendance"""
14460432,"""au9747cp""","""Jericho""","""Greer""",2,1,1,"""2019-01-16 14:00:00""","""2019-01-16 14:08:00""","""100 %""","""dsci""","""494""","""s7""","""Attendance"""
14460432,"""au9747cp""","""Jericho""","""Greer""",3,1,1,"""2019-01-18 14:00:00""","""2019-01-18 14:05:00""","""100 %""","""dsci""","""494""","""s7""","""Attendance"""
14460432,"""au9747cp""","""Jericho""","""Greer""",4,1,1,"""2019-01-23 14:00:00""","""2019-01-23 14:06:00""","""100 %""","""dsci""","""494""","""s7""","""Attendance"""
14460432,"""au9747cp""","""Jericho""","""Greer""",5,1,1,"""2019-01-25 14:00:00""","""2019-01-25 14:10:00""","""100 %""","""dsci""","""494""","""s7""","""Attendance"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
12630683,"""iv8164nd""","""Vinci""","""Alberich""",16,1,1,"""2019-03-04 11:01:00""","""2019-03-04 11:04:00""","""100 %""","""stat""","""491""","""s1""","""Attendance"""
12630683,"""iv8164nd""","""Vinci""","""Alberich""",17,1,1,"""2019-03-06 11:58:00""","""2019-03-06 12:02:00""","""100 %""","""stat""","""491""","""s1""","""Attendance"""
12630683,"""iv8164nd""","""Vinci""","""Alberich""",1,1,1,"""2019-01-14 11:03:00""","""2019-01-14 11:10:00""","""100 %""","""stat""","""491""","""s1""","""Attendance"""


## <font color="red"> Exercise 3.1.2 - Combine all `Practice Quizzes` files</font>

Now you try it. Combine all the `Practice Quiz` files in the `data/old_attendance_example` directory. Add a column with the path to each dataframe before combining them.

1. Use `glob` to get all the paths to the CSV files.
2. Use a list comprehension to read each CSV file, filter for `Practice Quiz` files, and add a column with the path.
3. Combine all the dataframes using `pl.concat`.
4. Extract the course information from the path and add it as a column.

In [38]:
# Your code here
(all_csvA :=
 glob('./data/old_attendance_example/**/*.csv', 
      recursive=True # Toggle comment and see what happens
     )
)


['./data/old_attendance_example\\dsci494s7\\Attendance Quiz - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 1 - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 2 - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 3 - User Attempts.csv',
 './data/old_attendance_example\\dsci494s7\\Practice Quiz - Module 4 - User Attempts.csv',
 './data/old_attendance_example\\stat180s18\\Attendance Quiz - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Attendance Quiz - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 1 - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 2 - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 3 - User Attempts.csv',
 './data/old_attendance_example\\stat491s1\\Practice Quiz - Module 4 - User Attempts.csv']

In [39]:
(all_tables :=
    [pl.read_csv(P)
       .with_columns(path = pl.lit(P))
     for P in all_csvs
     if 'Practice Quiz' in P
    ]
) >> map(obj.head(2))

[shape: (2, 11)
 ┌────────────┬──────────┬───────────┬──────────┬───┬────────────┬────────────┬─────────┬───────────┐
 │ Org        ┆ UserName ┆ FirstName ┆ LastName ┆ … ┆ Attempt_St ┆ Attempt_En ┆ Percent ┆ path      │
 │ Defined ID ┆ ---      ┆ ---       ┆ ---      ┆   ┆ art        ┆ d          ┆ ---     ┆ ---       │
 │ ---        ┆ str      ┆ str       ┆ str      ┆   ┆ ---        ┆ ---        ┆ str     ┆ str       │
 │ i64        ┆          ┆           ┆          ┆   ┆ str        ┆ str        ┆         ┆           │
 ╞════════════╪══════════╪═══════════╪══════════╪═══╪════════════╪════════════╪═════════╪═══════════╡
 │ 14460432   ┆ au9747cp ┆ Jericho   ┆ Greer    ┆ … ┆ 2019-01-28 ┆ 2019-01-28 ┆ 50 %    ┆ ./data/ol │
 │            ┆          ┆           ┆          ┆   ┆ 15:26:00   ┆ 15:30:00   ┆         ┆ d_attenda │
 │            ┆          ┆           ┆          ┆   ┆            ┆            ┆         ┆ nce_examp │
 │            ┆          ┆           ┆          ┆   ┆            ┆

In [40]:
(practice_quizzes :=
 pl.concat(all_tables)
).head()

Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent,path
i64,str,str,str,i64,i64,i64,str,str,str,str
14460432,"""au9747cp""","""Jericho""","""Greer""",1,10,20,"""2019-01-28 15:26:00""","""2019-01-28 15:30:00""","""50 %""","""./data/old_attendance_example\dsci494s7\Practice Quiz - Module 1 - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",1,19,20,"""2019-01-27 15:25:00""","""2019-01-27 15:34:00""","""95 %""","""./data/old_attendance_example\dsci494s7\Practice Quiz - Module 1 - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",2,11,20,"""2019-01-27 15:29:00""","""2019-01-27 15:33:00""","""55 %""","""./data/old_attendance_example\dsci494s7\Practice Quiz - Module 1 - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",3,9,20,"""2019-01-27 15:37:00""","""2019-01-27 15:38:00""","""45 %""","""./data/old_attendance_example\dsci494s7\Practice Quiz - Module 1 - User Attempts.csv"""
14460432,"""au9747cp""","""Jericho""","""Greer""",4,3,20,"""2019-01-27 15:43:00""","""2019-01-27 15:49:00""","""15 %""","""./data/old_attendance_example\dsci494s7\Practice Quiz - Module 1 - User Attempts.csv"""


In [41]:
pl.Config.set_fmt_table_cell_list_len(200) # Make the list column output longer
pl.Config.set_fmt_str_lengths(200) # Make string column output longer

(practice_quizzes_w_info :=
 practice_quizzes

.with_columns(path=pl.col('path').str.replace_all(r'\\', '/'))  # Replace \ with /
    .with_columns(path_split=pl.col('path').str.split('/'))
.with_columns(course_info = pl.col('path_split').list.get(3),
               file_name = pl.col('path_split').list.get(-1),
              )
  .with_columns(program = pl.col('course_info').str.slice(0,4),
                course_number = pl.col('course_info').str.slice(4, 3),
                section = pl.col('course_info').str.slice(7),
                quiz_type = pl.col('file_name').str.split(' ').list.get(0),
                )
 .drop(cs.starts_with('path', 'file') | cs.ends_with('info'))

)

Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent,program,course_number,section,quiz_type
i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,str
14460432,"""au9747cp""","""Jericho""","""Greer""",1,10,20,"""2019-01-28 15:26:00""","""2019-01-28 15:30:00""","""50 %""","""dsci""","""494""","""s7""","""Practice"""
14460432,"""au9747cp""","""Jericho""","""Greer""",1,19,20,"""2019-01-27 15:25:00""","""2019-01-27 15:34:00""","""95 %""","""dsci""","""494""","""s7""","""Practice"""
14460432,"""au9747cp""","""Jericho""","""Greer""",2,11,20,"""2019-01-27 15:29:00""","""2019-01-27 15:33:00""","""55 %""","""dsci""","""494""","""s7""","""Practice"""
14460432,"""au9747cp""","""Jericho""","""Greer""",3,9,20,"""2019-01-27 15:37:00""","""2019-01-27 15:38:00""","""45 %""","""dsci""","""494""","""s7""","""Practice"""
14460432,"""au9747cp""","""Jericho""","""Greer""",4,3,20,"""2019-01-27 15:43:00""","""2019-01-27 15:49:00""","""15 %""","""dsci""","""494""","""s7""","""Practice"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
18630821,"""ct6978kd""","""Ekstrom""","""Czech""",6,11,20,"""2019-02-25 11:32:00""","""2019-02-25 11:35:00""","""55 %""","""stat""","""491""","""s1""","""Practice"""
18630821,"""ct6978kd""","""Ekstrom""","""Czech""",7,10,20,"""2019-02-27 11:54:00""","""2019-02-27 12:02:00""","""50 %""","""stat""","""491""","""s1""","""Practice"""
18630821,"""ct6978kd""","""Ekstrom""","""Czech""",1,3,20,"""2019-02-22 11:01:00""","""2019-02-22 11:05:00""","""15 %""","""stat""","""491""","""s1""","""Practice"""
